In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
curruser = os.environ.get('USER')

_labdata = os.environ.get("LABDATA_PYSPARK")
sys.path.insert(0, _labdata)
os.chdir(_labdata)

if curruser in os.listdir("/opt/workspace/"):
    sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
    sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
    # sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata_v1.2/lib/'.format(user=curruser))
else:
    sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser))
    sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    # sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime as dt

from transliterate import translit

from lib.spark_connector import SparkConnector
from lib.sparkdb_loader import *
from lib.connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import Row

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from collections import ChainMap

from lib.config import *
from lib.tools import *

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))

def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

    return _typed_udf_wrapper

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None

# CONN_SCHEMA = 'sbx_team_digitcamp' #'sbx_t_team_cvm'


In [3]:
##-----------------------------------------
conn_schema = 'sbx_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
tbl = 'GA_SITE_ALL_PRODS'
out_table_name = 'GA_ALL_SCENARIOS_HIST'
##-----------------------------------------

def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

# @typed_udf(StringType())
def match_product(product_dict_nm):
    def get_product(product_norm, product_dict_nm):
        if product_norm is None:
            return None

        for product_nm in product_dict_nm:
            if product_nm in product_norm:
                return product_dict_nm[product_nm]
            else:
                return None

    return f.udf(lambda x: get_product(x,product_dict_nm))

In [5]:
sys.path.insert(0, '/opt/workspace/{user}/notebooks/ETL_EXAMPLES/ORA_TERA_ETL/ora_examples/src'.format(user=curruser))

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from etl import ETLORA

In [6]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
               dynamic_alloc=False,
               numofinstances=9,
               numofcores=8,
               executor_memory='35g',
               driver_memory='35g',
               kerberos_auth=True,
               process_label="SAS_EXPORT2HIVE"
               )

hive = sp.sql
print(sp.sc.version)

### Starting spark context. Run!
====================================================== 2021-09-16 ======================================================
# __init__ : begin
2.4.0.cloudera2


## Establish a Connection to Oracle WareHouse

In [9]:
#'MA_CMDM_MA_DEAL' #'AIST_INTERACTION_HISTORY' 'NRT_ALL_SCENARIOS_HIST'
ISKRA_TABLE = 'SME_CDM_MMB_OFFER_TO_INSIGHT'
user_tabs = 'user_tab_columns'

In [10]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect()
curs = db.cursor

In [14]:
query = \
'''
select /*+ parallel (12) */ inn, sbboluserid, id_product, scenario_id, funnel_rate, load_dt from GA_ALL_SCENARIOS_HIST 
where 1=1 and
scenario_id in ('SITEALLPRODS_01', 'STORIES_01') and 
load_dt = to_date('2020-06-30', 'yyyy-mm-dd')
'''

In [31]:
query=\
'''
select/*+ parallel (12) */
OFFER_ID
,INSIGHT_ID
,SAS_DYN_PAR_ID
,INPUT_NV_FLG
,EXCLD_FLG
,TO_CHAR(LOAD_DTTM, 'yyyy-mm-dd') as LOAD_DTTM
--distinct(load_dttm) 
from {}
where 1=1
and rownum < 300000
--order by load_dttm desc
'''.format(ISKRA_TABLE)
df = pd.read_sql(query, con=db.connection)

In [33]:
df.dtypes

OFFER_ID           int64
INSIGHT_ID         int64
SAS_DYN_PAR_ID    object
INPUT_NV_FLG       int64
EXCLD_FLG          int64
LOAD_DTTM         object
dtype: object

In [5]:
query = '''
(select /*+ parallel (12) */
        * --CALL_INCOMING_CALL, ACTION_ITEM_PRODUCT_ID
FROM {}
where 1=1
--and CALL_INCOMING_CALL = 0
and rownum < 400000
--and 
--( (ACTION_ITEM_PRODUCT_ID is not Null) 
--  and 
--  (ACTION_ITEM_PRODUCT_ID <> 'null')
--)
)
'''.format(ISKRA_TABLE)

In [20]:
df = pd.read_sql(query, con=db.connection)

In [55]:
df.sort_values(by=['ACCT_SIGNED_DT'], ascending=False).head(50)

,AGREEMENT_ID,INN,KPP,ACCT_SIGNED_DT,CONTRACT_EXPIRATION_DT,ACCT_CLOSE_DT,FIN_AGRMNT_TYPE_DESC,ISO_CRNCY_CD,CRNCY_CD,AGREEMENT_NUM,ANALYTIC_CLOSE_DT
357,22532127966,6910013690,691001001,2021-04-22,2021-05-11,NaT,Депозит юридического лица,RUR,810,8607233211.ПУ00,NaT
2231,22488513344,7702787952,770501001,2021-04-21,2021-04-23,NaT,Сделка с неснижаемым остатком,RUR,810,9038220283,NaT
2110,22485214457,6829052587,682901001,2021-04-21,2021-06-15,NaT,Депозит юридического лица,RUR,810,8594218859.00,NaT
4749,22481148246,622804074220,None,2021-04-21,2024-04-19,NaT,Кредитная линия,RUR,810,8606Q65FFSOS2Q0SQ0QF9B,NaT
3167,22484699703,2926010874,290101001,2021-04-21,2021-04-28,NaT,Депозит юридического лица,RUR,810,8637231234.ПУ00,NaT
3478,22439812594,5403175098,540301001,2021-04-20,2021-05-03,NaT,Сделка с неснижаемым остатком,RUR,810,8047228013.ПУ,NaT
1528,22383556926,290207058338,None,2021-04-19,2021-04-26,NaT,Сделка с неснижаемым остатком,RUR,810,8637227163.ПУ,NaT
3883,22391833256,9701048328,770101001,2021-04-19,2021-10-25,NaT,Депозит юридического лица,RUR,810,9038213061.00,NaT
779,22388965166,7302007037,732901001,2021-04-19,2021-04-20,2021-04-20,Сделка с неснижаемым остатком,RUR,810,8588212334,2021-04-20
403,22334974052,7721489260,772801001,2021-04-16,2021-04-23,NaT,Депозит юридического лица,RUR,810,9040223166.ПУ00,NaT


In [11]:
query = '''
select /*+ parallel (10) */
column_name, data_type||
case
when data_precision is not Null and nvl(data_scale,0)>0 then '('||data_precision||', '||data_scale||')'
when data_precision is not Null and nvl(data_scale,0)=0 then '('||data_precision||')'
when data_precision is Null and data_scale is not Null then '(*, '||data_scale||')'
when char_length>0 then '('||char_length||', '||case char_used
                                                     when 'B' then 'Byte'
                                                     when 'C' then 'Char'
                                                     else null
                                                end||')'
end||decode(nullable, 'N', 'NOT NULL') as data_type
FROM {}
where table_name = '{}'
'''.format(user_tabs, ISKRA_TABLE)
df = pd.read_sql(query, con=db.connection)

In [12]:
df.head(65)

,COLUMN_NAME,DATA_TYPE
0,OFFER_ID,NUMBER
1,INSIGHT_ID,NUMBER
2,SAS_DYN_PAR_ID,NUMBER
3,INPUT_NV_FLG,NUMBER(9)
4,EXCLD_FLG,NUMBER(9)
5,LOAD_DTTM,DATE


## FIRST: Fetching a small part of dataframe

In [13]:
query = '''
(select /*+ parallel (8) */
          *
FROM {}
where rownum < 500)
'''.format(ISKRA_TABLE)

In [14]:
sdf = sp.get_oracle(OracleDB('iskra4'), query)

In [15]:
sdf.show()

,OFFER_ID,INSIGHT_ID,SAS_DYN_PAR_ID,INPUT_NV_FLG,EXCLD_FLG,LOAD_DTTM
0,8663212874.0000000000,7586214121.0000000000,None,0,0,2021-05-05
1,8658985001.0000000000,7586716858.0000000000,None,0,0,2021-05-05
2,8663227825.0000000000,7587163220.0000000000,None,0,0,2021-05-05
3,8664441404.0000000000,7584921469.0000000000,None,0,0,2021-05-05
4,8682260600.0000000000,7585964789.0000000000,None,0,0,2021-05-05
5,8666100602.0000000000,7586330242.0000000000,None,0,0,2021-05-05
6,8655383267.0000000000,7584629716.0000000000,None,0,0,2021-05-05
7,8665587950.0000000000,7584722207.0000000000,None,0,0,2021-05-05
8,8663496639.0000000000,7587803137.0000000000,None,0,0,2021-05-05
9,8660437303.0000000000,7585679140.0000000000,None,0,0,2021-05-05


# Check types in advance

In [30]:
etl = ETLORA()

### Create a dummy dataframe in Hive

In [32]:
tbls =\
[
    "MA_TMP_MMB_OFFER_PROC_COE",
    "ma_tmp_mmb_offer_proc_exclude",
    "SME_CDM_ma_mmb_offer_history",
    "S_cdm.ma_mmb_stop_customer_history",
    "ma_dict.ma_mmb_conv_coef_dict",
    "S_cdm.ma_mmb_stop_offer_history"
]

In [15]:
ISKRA_TABLE = tbls[1] #'AIST_INTERACTION_HISTORY' #'MA_PRODUCT_DICT' #'MA_MMB_OFFER_NONTOP' #'OFFER_PRIORITY' #'MA_MMB_OFFER_NONTOP' #'MA_TASK' #'AIST_INTERACTION_HIST'
hive_out_tbl = ISKRA_TABLE

In [33]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect()
curs = db.cursor

In [34]:
my_sql = '''
select /*+ parallel (10) */
              * 
FROM {}
where rownum < 10000
'''.format(ISKRA_TABLE)

In [35]:
df = pd.read_sql(my_sql, con=db.connection)

In [36]:
res, cols = etl.get_data_from_oracle(curs, my_sql)
# df = etl.get_df_from_oracle(my_sql, cols)
cols

['INN',
 'SBBOLUSERID',
 'COMMONSEGMENTOUID',
 'ID_CRM',
 'SCENARIO_ID',
 'CHANNEL',
 'ID_PRODUCT',
 'MADEAL_PRODUCT_NORM',
 'MADEAL_PROD_TYPE_NAME',
 'MADEAL_APPL_CORE_TXT',
 'MADEAL_COMPLETE_DT',
 'HASPRODUCT',
 'ESSENSE',
 'RETURNCNT',
 'NUMBEROFSTEPS',
 'FUNNEL_RATE',
 'MINSESSIONSTARTTIME',
 'MAXSESSIONSTARTTIME',
 'LOAD_DT',
 'SESSIONDATE']

In [7]:
import subprocess

In [13]:
subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format(ISKRA_TABLE.lower())])
sql = "drop table if exists {}.{} purge".format(conn_schema,ISKRA_TABLE)
hive.sql(sql)

DataFrame[]

In [14]:
# print("### Recreate target table in Hive")

typesmap_rdd, typesmap_pd = etl.create_table_hive(curs, sp, my_sql, conn_schema, ISKRA_TABLE)

print("### Consecutively fetch buckets from Oracle and insert ones into Hive table")

query = '''
select /*+ parallel (12) */
          *
FROM {}
'''.format(ISKRA_TABLE)

# db.connect()
curs = db.cursor
curs.execute(query)

hive = sp.sql
while True:
    df = etl.get_df_from_oracle(curs, cols, size=500000)
    print('###--> Get bucket of size {} from Oracle'.format(df.shape))
    if df.shape[0]!=0:
        etl.update_table_hive(sp, df, typesmap_rdd, typesmap_pd, conn_schema, ISKRA_TABLE)
        print('###--> {} records has been inserted...'.format(df.shape[0]))
        del df
    else:
        break

db.close()

### Consecutively fetch buckets from Oracle and insert ones into Hive table
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has been inserted...
###--> Get bucket of size (500000, 24) from Oracle
###--> 500000 records has b

## Using JDBC

In [7]:
tbls =\
[
    "MA_MMB_CONV_COEF_DICT" 
#     "MA_TMP_MMB_OFFER_PROC_COE",
#     "ma_tmp_mmb_offer_proc_exclude",
#     "SME_CDM_ma_mmb_offer_history",
#     "ma_mmb_stop_customer_history",
#     "ma_mmb_conv_coef_dict",
#     "ma_mmb_stop_offer_history"
]


In [68]:
# ISKRA_TABLE = tbls[5]

In [8]:
for tbl in tbls:
    ISKRA_TABLE = tbl
    print("Working with table: {}".format(ISKRA_TABLE))
    my_sql = '''
    (select /*+ parallel (12) */
                  * 
    FROM {}
    where rownum < 20000)
    '''.format(ISKRA_TABLE)

    sdf = sp.get_oracle(OracleDB('iskra4'), my_sql, BATCH_SIZE=10000)

    hive.sql("drop table if exists {new_schema}.{tbl} purge".format(new_schema="sbx_team_digitcamp", 
                                                                    tbl=ISKRA_TABLE))    
    insert = ', '.join(["{} {}".format(col, _type) for col, _type in sdf.dtypes])

    hive.sql('''create table {schema}.{tbl} (
                                             {fields}
                                                )
             '''.format(schema="sbx_team_digitcamp",
                        tbl=ISKRA_TABLE, 
                        fields=insert)
                       )

    sdf = sp.get_oracle(OracleDB('iskra4'), ISKRA_TABLE, BATCH_SIZE=600000)
    sdf.registerTempTable("tmp_res")
    hive.sql("insert into sbx_team_digitcamp.{} select * from tmp_res".format(ISKRA_TABLE))
    print("End of batch processing. Switch to the next db")

Working with table: MA_MMB_CONV_COEF_DICT
End of batch processing. Switch to the next db


In [8]:
for tbl in tbls:
    ISKRA_TABLE = tbl
    subprocess.call(['hdfs', 'dfs', '-rm', '-r', '-f', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format(ISKRA_TABLE.lower())])
    sql = "drop table if exists {}.{} purge".format(conn_schema,ISKRA_TABLE)
    hive.sql(sql)

In [73]:
sdf = hive.table("{}.{}".format("sbx_team_digitcamp", ISKRA_TABLE))
sdf.count()

501277686

## DROP tables on ISKRA server 

In [5]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect()
curs = db.cursor

In [6]:
for tbl in tbls:
    sql = "DROP TABLE {}".format(tbl)
    db.cursor.execute(sql)
    db.connection.commit()
    print('Dropping table {}... Done!'.format(tbl))

Dropping table MA_TMP_MMB_OFFER_PROC_COE... Done!
Dropping table ma_tmp_mmb_offer_proc_exclude... Done!
Dropping table SME_CDM_ma_mmb_offer_history... Done!
Dropping table ma_mmb_stop_customer_history... Done!
Dropping table ma_mmb_conv_coef_dict... Done!
Dropping table ma_mmb_stop_offer_history... Done!
